In [2]:
import pandas as pd

# ======================================
# Path
# ======================================
csv_path = '../paper2/total/true_detections_final.csv'

# ======================================
# Read file
# ======================================
df = pd.read_csv(csv_path)

# Sort by Source ID
df = df.sort_values('Source_ID')

# Take first 10 rows
sample = df.head(10)

# Select only needed columns (order matters)
sample = sample[
    ['Source_ID', 'RA', 'DEC', 'RM', 'E_RM', 'fracPol',
     'S_reffreq', 'RMS', 'F1', 'F2']
]

# ======================================
# Formatting
# ======================================
sample['RA']        = sample['RA'].map(lambda x: f"{x:.5f}")
sample['DEC']       = sample['DEC'].map(lambda x: f"{x:.5f}")
sample['RM']        = sample['RM'].map(lambda x: f"{x:.2f}")
sample['E_RM']      = sample['E_RM'].map(lambda x: f"{x:.2f}")
sample['fracPol']   = sample['fracPol'].map(lambda x: f"{x:.2f}")
sample['S_reffreq'] = sample['S_reffreq'].map(lambda x: f"{x:.2f}")

# Convert RMS from Jy to μJy BEFORE formatting
sample['RMS'] = (sample['RMS'] * 1e6).map(lambda x: f"{x:.1f}")

# ======================================
# Print LaTeX table
# ======================================
print(r"""
\begin{table*}
\centering
\small
\setlength{\tabcolsep}{3pt}
\renewcommand{\arraystretch}{1.1}
\caption{Example rows from the machine-readable catalog of true Faraday-rotation detections,
sorted by Source ID. Units are given in the header row.}
\label{tab:true_detections_final_sample}
\begin{tabular}{lccccccccc}
\toprule
Source ID & RA & Dec & RM & dRM & fracPol & $S_{\mathrm{ref}}$ & RMS & F1 & F2 \\
          & deg & deg & rad m$^{-2}$ & rad m$^{-2}$ & \% & mJy & $\mu$Jy & flag & flag \\
\midrule
""")

for _, row in sample.iterrows():
    print(
        f"{row['Source_ID']} & "
        f"{row['RA']} & "
        f"{row['DEC']} & "
        f"{row['RM']} & "
        f"{row['E_RM']} & "
        f"{row['fracPol']} & "
        f"{row['S_reffreq']} & "
        f"{row['RMS']} & "
        f"{int(row['F1'])} & "
        f"{int(row['F2'])} \\\\"
    )

print(r"""
\bottomrule
\end{tabular}
\end{table*}
""")



\begin{table*}
\centering
\small
\setlength{\tabcolsep}{3pt}
\renewcommand{\arraystretch}{1.1}
\caption{Example rows from the machine-readable catalog of true Faraday-rotation detections,
sorted by Source ID. Units are given in the header row.}
\label{tab:true_detections_final_sample}
\begin{tabular}{lccccccccc}
\toprule
Source ID & RA & Dec & RM & dRM & fracPol & $S_{\mathrm{ref}}$ & RMS & F1 & F2 \\
          & deg & deg & rad m$^{-2}$ & rad m$^{-2}$ & \% & mJy & $\mu$Jy & flag & flag \\
\midrule

VCPMC J031923.8+310143 & 49.84941 & 31.02869 & 33.78 & 0.49 & 5.48 & 0.04 & 110.8 & 0 & 0 \\
VCPMC J031927.6+312115 & 49.86532 & 31.35421 & 58.60 & 1.46 & 7.51 & 0.01 & 36.5 & 0 & 0 \\
VCPMC J032003.0+310733 & 50.01252 & 31.12604 & 75.95 & 0.12 & 5.09 & 0.11 & 147.1 & 0 & 1 \\
VCPMC J032021.4+313824 & 50.08925 & 31.64013 & 47.79 & 0.49 & 6.89 & 0.02 & 47.2 & 0 & 0 \\
VCPMC J032027.7+315638 & 50.11581 & 31.94413 & 44.81 & 1.05 & 15.68 & 0.00 & 30.8 & 0 & 0 \\
VCPMC J032044.6+314601 & 50.186

In [3]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u

# ======================================
# INPUT / OUTPUT
# ======================================
input_csv        = "../paper2/total/true_detections_final.csv"
output_mrt       = "../paper2/total/true_detections_final_mrt.txt"
output_data_only = "../paper2/total/true_detections_final_data_only.txt"

# ======================================
# IAU-COMPLIANT SOURCE NAME (STRICT TRUNCATION)
# Format: VCPMC JHHMMSS.s+DDMMSS
# (truncated, not rounded)
# ======================================
def make_vcpmc_name(ra_deg, dec_deg, prefix="VCPMC"):
    coord = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg, frame="icrs")

    # RA: truncate to 0.1 s
    ra_h = coord.ra.hms.h
    ra_m = coord.ra.hms.m
    ra_s = coord.ra.hms.s
    ra_s_trunc = np.floor(ra_s * 10.0) / 10.0
    ra_str = f"{int(ra_h):02d}{int(ra_m):02d}{ra_s_trunc:04.1f}"  # HHMMSS.s

    # Dec: truncate to integer arcsec
    dec_sign = "+" if coord.dec.deg >= 0 else "-"
    dec_d = abs(coord.dec.dms.d)
    dec_m = coord.dec.dms.m
    dec_s = coord.dec.dms.s
    dec_s_trunc = int(np.floor(dec_s))
    dec_str = f"{dec_sign}{int(dec_d):02d}{int(dec_m):02d}{dec_s_trunc:02d}"  # +DDMMSS

    return f"{prefix} J{ra_str}{dec_str}"


# ======================================
# LOAD
# ======================================
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# Clean and regenerate Source_ID (recommended: guarantees truncation + new prefix)
if not {"RA", "DEC"}.issubset(df.columns):
    raise KeyError("CSV must include RA and DEC columns (degrees) to regenerate Source_ID safely.")

df["Source_ID"] = df.apply(lambda r: make_vcpmc_name(float(r["RA"]), float(r["DEC"]), prefix="VCPMC"), axis=1)

# Optional: sort by Source_ID (now VCPMC…)
df = df.sort_values("Source_ID")

# ======================================
# COLUMN ORDER FOR MRT
# ======================================
columns = [
    "Source_ID",
    "RA",
    "DEC",
    "RM",
    "E_RM",
    "fracPol",
    "S_reffreq",
    "RMS",
    "F1",
    "F2"
]

# Keep only columns that exist (avoids crashing if a column is missing)
missing = [c for c in columns if c not in df.columns]
if missing:
    raise KeyError(f"Missing required columns in CSV: {missing}")

# ======================================
# UNIT CONVERSIONS
# ======================================
# RMS: Jy -> µJy (if RMS currently in Jy)
df["RMS"] = df["RMS"] * 1e6

# S_reffreq: Jy -> mJy (if S_reffreq currently in Jy)
df["S_reffreq"] = df["S_reffreq"] * 1e3

# ======================================
# HEADER ROWS
# ======================================
labels = columns

# Units
units = []
for col in columns:
    if col in ["RA", "DEC"]:
        units.append("deg")
    elif col in ["RM", "E_RM"]:
        units.append("rad/m2")
    elif col == "fracPol":
        units.append("%")
    elif col == "S_reffreq":
        units.append("mJy")
    elif col == "RMS":
        units.append("uJy")
    else:
        units.append("---")

# Explanations
explanations = []
for col in columns:
    if col == "Source_ID":
        explanations.append("IAU-compliant source name (VCPMC JHHMMSS.s+DDMMSS; coordinates truncated, not rounded)")
    elif col == "RA":
        explanations.append("Right ascension (J2000) in degrees")
    elif col == "DEC":
        explanations.append("Declination (J2000) in degrees")
    elif col == "RM":
        explanations.append("Rotation measure")
    elif col == "E_RM":
        explanations.append("Uncertainty in rotation measure")
    elif col == "fracPol":
        explanations.append("Fractional polarization at reference frequency")
    elif col == "S_reffreq":
        explanations.append("Total Stokes I flux density at reference frequency")
    elif col == "RMS":
        explanations.append("Local RMS noise at reference frequency")
    elif col == "F1":
        explanations.append("Flag: source lies on diffuse Galactic polarized emission region (1=yes, 0=no)")
    elif col == "F2":
        explanations.append("Flag: double-peaked Faraday spectrum (1=yes, 0=no)")
    else:
        explanations.append("")

# ======================================
# AAS FORMAT CODES (UPDATED FOR NEW Source_ID)
# New ID length:
#   "VCPMC J" (6) + "HHMMSS.s" (8) + "+DDMMSS" (7) = 21 chars
# Example: VCPMC J034012.7+314512
# ======================================
formats = []
for col in columns:
    if col == "Source_ID":
        formats.append("A21")
    elif col in ["RA", "DEC"]:
        formats.append("F10.5")
    elif col in ["RM", "E_RM"]:
        formats.append("F9.2")
    elif col == "fracPol":
        formats.append("F7.2")
    elif col == "S_reffreq":
        formats.append("F9.2")   # slightly wider is safer after Jy->mJy
    elif col == "RMS":
        formats.append("F9.2")
    elif col in ["F1", "F2"]:
        formats.append("I1")
    else:
        formats.append("F8.2")

# Python fixed-width formats (match widths above)
py_formats = []
for col in columns:
    if col == "Source_ID":
        py_formats.append("s21")
    elif col in ["RA", "DEC"]:
        py_formats.append("f10.5")
    elif col in ["RM", "E_RM"]:
        py_formats.append("f9.2")
    elif col == "fracPol":
        py_formats.append("f7.2")
    elif col == "S_reffreq":
        py_formats.append("f9.2")
    elif col == "RMS":
        py_formats.append("f9.2")
    elif col in ["F1", "F2"]:
        py_formats.append("i1")
    else:
        py_formats.append("f8.2")

# ======================================
# HELPER: FIXED-WIDTH FORMATTING
# ======================================
def format_fixed(x, fmt):
    if pd.isna(x):
        spec = fmt[0]
        if spec in ["f", "i"]:
            width = int(fmt[1:].split(".")[0]) if "." in fmt[1:] else int(fmt[1:])
            return "".rjust(width)
        if spec == "s":
            width = int(fmt[1:])
            return "".rjust(width)
        return ""

    spec = fmt[0]
    if spec == "s":
        width = int(fmt[1:])
        return f"{str(x):>{width}s}"
    if spec == "i":
        width = int(fmt[1:])
        return f"{int(x):>{width}d}"
    if spec == "f":
        w, p = fmt[1:].split(".")
        width = int(w)
        precision = int(p)
        return f"{float(x):>{width}.{precision}f}"
    return str(x)

# ======================================
# FORMAT DATA ROWS
# ======================================
formatted_rows = []
for _, r in df.iterrows():
    formatted_rows.append(" ".join(format_fixed(r[col], fmt) for col, fmt in zip(columns, py_formats)))

# ======================================
# WRITE MRT (HEADER + DATA)
# ======================================
with open(output_mrt, "w") as f:
    f.write(" ".join(labels) + "\n")
    f.write(" ".join(units) + "\n")
    f.write(" ".join(explanations) + "\n")
    f.write(" ".join(formats) + "\n")
    for line in formatted_rows:
        f.write(line + "\n")

print(f"✅ Saved MRT file to: {output_mrt}")

# ======================================
# WRITE DATA-ONLY
# ======================================
with open(output_data_only, "w") as f:
    for line in formatted_rows:
        f.write(line + "\n")

print(f"✅ Saved data-only file to: {output_data_only}")

# ======================================
# FORMAT CODE STRING FOR AAS SUBMISSION
# ======================================
print("\n👉 Copy this into the AAS 'format code' box:\n")
print(" ".join(formats))


✅ Saved MRT file to: ../paper2/total/true_detections_final_mrt.txt
✅ Saved data-only file to: ../paper2/total/true_detections_final_data_only.txt

👉 Copy this into the AAS 'format code' box:

A21 F10.5 F10.5 F9.2 F9.2 F7.2 F9.2 F9.2 I1 I1
